In [2]:
from darkflow.net.build import TFNet
import cv2

options = {"model": "cfg/yolo-voc.cfg", "load": "bin/yolo-voc.weights", "threshold": 0.2}
tfnet = TFNet(options)

Parsing ./cfg/yolo-voc.cfg
Parsing cfg/yolo-voc.cfg
Loading bin/yolo-voc.weights ...
Successfully identified 202704260 bytes
Finished in 0.022908449172973633s
Model has a VOC model name, loading VOC labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 128)
 Lo

In [11]:
# To do the bulk of the work for us, data manipulation and image control
import numpy as np
import cv2
# To hold the functions created for this project
from featureprocesser import *

In [16]:
global frame_buffer
frame_buffer = []
def yolo_pipeline(image):
    
    # results is a list of dictionaries. 
    # It contains all boxes with enough confidence, as set above
    results = tfnet.return_predict(image)
    
    boxes = []
    for result in results: 
        if ['label'] == 'car':   
            box = [[result['topleft']['x'],result['topleft']['y']],[result['bottomright']['x'], result['bottomright']['y']]]
            boxes.append(box)
     
    # Once we have a full buffer, create average boxes from those frames
    # This smoothes the boxes in the video and removes random detections
    frame_buffer.append(boxes)
    if len(frame_buffer) > 5:
        frame_buffer.pop(0)
        
        heatmap = np.zeros_like(image[:,:,0])
        for frame in frame_buffer:
            heatmap = add_heat(heatmap, frame)
        heatmap = apply_threshold(heatmap, threshold = 4)
        labels = label(heatmap)
        
        boxes = calculate_labeled_boxes(image, labels)
    
    final_image = draw_boxes(image, boxes)
    
    return final_image
        
    
    

In [20]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

vid_output = './project_video_out.mp4'
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(vid_output))

In [18]:
vid_output = 'project_video_out.mp4'
clip = VideoFileClip('./project_video.mp4')
vid_clip = clip.fl_image(lambda x: yolo_pipeline(x))
vid_clip.write_videofile(vid_output, audio=False)

[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4



 12%|█▏        | 152/1261 [02:53<21:01,  1.14s/it]


 24%|██▍       | 303/1261 [05:48<17:59,  1.13s/it]


 36%|███▌      | 454/1261 [08:41<15:27,  1.15s/it]


 48%|████▊     | 605/1261 [11:35<12:36,  1.15s/it]


...

 72%|███████▏  | 907/1261 [17:34<06:45,  1.14s/it]


 84%|████████▍ | 1057/1261 [20:27<03:55,  1.15s/it]


 96%|█████████▌| 1205/1261 [23:15<01:03,  1.13s/it]


100%|█████████▉| 1260/1261 [24:18<00:01,  1.13s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

